In [ ]:
%run main.py

start: 2021-07-13 16:09:47.124214


In [ ]:
import pickle
import networkx
import random
import pandas as pd
import networkx as nx
import folium
from folium.plugins import HeatMap

import python.display as dp
import python.voxels as voxels
project_folder = "monresovelo"

In [2]:
with open("files/"+project_folder+"/city_graphs/city.ox",'rb') as infile:
    G = pickle.load(infile)

In [3]:
for v in G:
    for v_n in G[v]:
         G[v][v_n][0]['w']=1

In [4]:
node_list = []
for i in G.nodes:
    node_list.append(i)

In [5]:
list_od = []
list_sp_w = []
list_sp_l = []
i=0
error=0
while (i<100):
    d_node = random.choice(node_list)
    f_node = d_node
    while(f_node == d_node):
        f_node = random.choice(node_list)
    try:
        sp_w = nx.shortest_path(G, source=d_node, target=f_node, weight="w")
        sp_l = nx.shortest_path(G, source=d_node, target=f_node, weight="length")
        
        route_coord = [[G.nodes[x]["y"], G.nodes[x]["x"]] for x in sp_w]
        route_coord = [x + [i] for x in route_coord]
        for c in route_coord:
            list_sp_w.append(c)
        
        route_coord = [[G.nodes[x]["y"], G.nodes[x]["x"]] for x in sp_l]
        route_coord = [x + [i] for x in route_coord]
        for c in route_coord:
            list_sp_l.append(c)
              
        list_od.append([d_node, f_node])
        i+=1
    except nx.NetworkXNoPath:
        print("No path...")
df_sp_w=pd.DataFrame(list_sp_w, columns=["lat", "lon", "route_num"], dtype=object)
df_sp_l=pd.DataFrame(list_sp_l, columns=["lat", "lon", "route_num"], dtype=object)

No path...
No path...
No path...
No path...


In [6]:
df_sp_w

,lat,lon,route_num
0,45.5105,-73.5728,0
1,45.5114,-73.5748,0
2,45.5118,-73.5745,0
3,45.5121,-73.5742,0
4,45.5127,-73.5736,0
...,...,...,...
3099,45.5121,-73.5742,98
3100,45.5127,-73.5736,98
3101,45.504,-73.5783,99
3102,45.504,-73.5785,99


In [7]:
tab_routes_voxels_sp_w, tab_routes_voxels_sp_w_global, dict_voxels_sp_w = voxels.generate_voxels(
    df_sp_w, df_sp_w.iloc[0]["route_num"], df_sp_w.iloc[-1]["route_num"])

tab_routes_voxels_sp_l, tab_routes_voxels_sp_l_global, dict_voxels_sp_l = voxels.generate_voxels(
    df_sp_l, df_sp_l.iloc[0]["route_num"], df_sp_l.iloc[-1]["route_num"])


In [8]:
if(display):     
    tab = []
    for key in dict_voxels_sp_w:
        vox_str = key.split(";")
        vox_int = [int(vox_str[0]), int(vox_str[1])]
        vox_pos = voxels.get_voxel_points(vox_int, 0)
        tab.append([vox_pos[0][0], vox_pos[0][1], dict_voxels_sp_w[key]["cyclability_coeff"]])

    df = pd.DataFrame(tab, columns=["lat", "lon", "Cyclability coefficient"])
    
    map = folium.Map(location=[df.iloc[0]["lat"],df.iloc[0]["lon"]], zoom_start=11,tiles = 'Stamen Toner')
    HeatMap(data=df.values.tolist(), max_zoom=13, radius=9, blur = 1, min_opacity = 0, max_val = 1).add_to(map)
map

In [9]:
if(display):     
    tab = []
    for key in dict_voxels_sp_l:
        vox_str = key.split(";")
        vox_int = [int(vox_str[0]), int(vox_str[1])]
        vox_pos = voxels.get_voxel_points(vox_int, 0)
        tab.append([vox_pos[0][0], vox_pos[0][1], dict_voxels_sp_l[key]["cyclability_coeff"]])

    df = pd.DataFrame(tab, columns=["lat", "lon", "Cyclability coefficient"])
    
    map = folium.Map(location=[df.iloc[0]["lat"],df.iloc[0]["lon"]], zoom_start=11,tiles = 'Stamen Toner')
    HeatMap(data=df.values.tolist(), max_zoom=13, radius=9, blur = 1, min_opacity = 0, max_val = 1).add_to(map)
map